# What are inner functions?

- We're able to define functions inside another

In [6]:
def outer_func():
    # do something
    def inner_func():
        pass
    return inner_func()

- In this example, we'll have the local scope of `inner_func` embedded inside the local scope of `outer_func`

![](images/nested_scopes_3.PNG)

- Both `inner_func` and `outer_func` have access to both the global and built-in scopes
    - Furthermore, `inner_func` has access to the scope of `outer_func`
        - To `inner_func`, the scope is `outer_func` is **neither local nor global**
            - It's called a **non-local scope**

### Examples

**Example 1**

- We'll create a new module containing the following:

```python
def outer_func():
    a = 10
    def inner_func():
        print(a)
    
    inner_func()

outer_func()
```

In [7]:
code = """
def outer_func():
    a = 10
    def inner_func():
        print(a)

    inner_func()

outer_func()
"""

In [8]:
with open('module_example_1.py', 'w') as f: 
    f.write(code) 

In [10]:
%run module_example_1

10


- As we can see, when we called `outer_func`, it:
    1. Defined `a` in its local scope
    2. Defined `inner_func`
        - Since `a` was never defined in the local scope of `inner_func`, it recognized `a` from the non-local scope of `outer_func`
    3. Called `inner_func`, which printed `a`

**Example 2**

- Let's consider a similar module:

```python
a = 10

def outer_func():
    def inner_func():
        print(a)
    
    inner_func()

outer_func()
```

- The difference between this example and the previous is where `a` is defined
    - Here, it's in the global scope instead of the local scope of `outer_func`

In [11]:
code = """
a = 10

def outer_func():
    def inner_func():
        print(a)

    inner_func()

outer_func()
"""

In [12]:
with open('module_example_2.py', 'w') as f: 
    f.write(code) 

In [16]:
%run module_example_2

10


- As we can see, the correct value was printed
    - The code:
        1. Didn't find `a` in the local scope of `inner_func`
        2. Didn't find `a` in the non-local scope of `outer_func`
        3. Found `a` in the global scope

**Example 3**

- This time, we'll define the global variable `a` inside the local scope of `inner_func`

```python
def outer_func():
    def inner_func():
        global a
        a = 'hello'
        print(a)

    inner_func()

outer_func()
print(a)
```

In [18]:
code = """
def outer_func():
    def inner_func():
        global a
        a = 'hello'
        print(a)

    inner_func()

outer_func()
print(a)
"""

In [19]:
with open('module_example_3.py', 'w') as f: 
    f.write(code) 

In [20]:
%run module_example_3

hello
hello


# Can we modify non-local variables?

In [21]:
def outer_func():
    x = 'hello'
    
    def inner_func():
        x = 'python'
        
    inner_func()
    
    print(x)

In [22]:
outer_func()

hello


- When `outer_func` is compiled, `x` is recognized as a local variable
    - When `inner_func` is compiled, its own version `x` is recognized as a local variable (for `inner_func`)

- This means that when we call `outer_func`, the `x` in the `inner_func` scope is recognized to be **different** from the `x` in the non-local scope of `outer_func`
    - Even though we call `inner_func` inside `outer_func`, once it is finished (while still inside the function), its scope is deleted
        - Therefore, by the time `print(x)` runs, the `x` inside the `inner_func` scope is already gone

- The only way to modify non-local variables is by explicity referencing it
    - We use the `nonlocal` keyword

In [23]:
def outer_func():
    x = 'hello'
    
    def inner_func():
        nonlocal x
        x = 'python'
        
    inner_func()
    
    print(x)

In [24]:
outer_func()

python


- This time, instead of creating a local variable for `x` in the scope of `inner_func`, we simply reference the variable in the non-local scope

- **Note**: when we use the `nonlocal` keyword, Python will look in all the scopes **except the global scope**

In [25]:
def outer():
    x = 'hello'
    
    def inner_1():
        
        def inner_2():
            nonlocal x
            x = 'python'
        
        inner_2()
        
    inner_1()
    print(x)

In [26]:
outer()

python


- As we can see, `x` was overwritten

![](images/non_local_scope.PNG)

In [27]:
def outer():
    x = 'hello'
    
    def inner_1():
        x = 'python'
        def inner_2():
            nonlocal x
            x = 'monty'
        print(f'inner (before): {x}')
        inner_2()
        print(f'inner (after): {x}')
        
    inner_1()
    print(f'outer (final): {x}')

In [28]:
outer()

inner (before): python
inner (after): monty
outer (final): hello


![](images/non_local_scope_2.PNG)

In [29]:
def outer():
    x = 'hello'
    
    def inner_1():
        nonlocal x
        x = 'python'
        def inner_2():
            nonlocal x
            x = 'monty'
        print(f'inner (before): {x}')
        inner_2()
        print(f'inner (after): {x}')
        
    inner_1()
    print(f'outer (final): {x}')

In [30]:
outer()

inner (before): python
inner (after): monty
outer (final): monty


![](images/non_local_scope_3.PNG)

In [31]:
x = 100

def outer():
    x = 'python'
    
    def inner_1():
        nonlocal x
        x = 'monty'
        def inner_2():
            global x
            x = 'hello'
        print(f'inner (before): {x}')
        inner_2()
        print(f'inner (after): {x}')
        
    inner_1()
    print(f'outer (final): {x}')

In [32]:
outer()

inner (before): monty
inner (after): monty
outer (final): monty


![](images/non_local_scope_4.PNG)